In [3]:
import numpy as np
import scipy.misc
import lmdb
import os
import sys
from struct import *

caffe_root='/usr/local/google/home/zyan/proj/caffe_private_recurrent_2d/'
sys.path.append(caffe_root + 'python')
from caffe import *
sys.path.append(caffe_root + 'python/caffe/proto')
import caffe_pb2

In [4]:
data_dir='/usr/local/google/home/zyan/data/mnist/'
train_img_data='train-images.idx3-ubyte'
train_img_label='train-labels.idx1-ubyte'
test_img_data='t10k-images.idx3-ubyte'
test_img_label='t10k-labels.idx1-ubyte'

caffe_data_dir=caffe_root+'data/mnist_semantic_segmentation/'
caffe_example_dir=caffe_root+'examples/mnist_semantic_segmentation/'
train_img_save_dir='train_images/'
test_img_save_dir='test_images/'

In [3]:
''' save into individual mnist images
'''
def extract_images_and_labels(img_data_file, img_label_file, img_save_path, out_label_file):
    if not os.path.exists(img_save_path):
        os.mkdir(img_save_path)
        
    
    with open(img_data_file,'rb') as f:
        magic_num_str = f.read(4)
        magic_num = (unpack('>i',magic_num_str))[0]
        num_img_str = f.read(4)
        num_img=(unpack('>i',num_img_str))[0]
        num_row_str=f.read(4)
        num_row=(unpack('>i',num_row_str))[0]
        num_col_str=f.read(4)
        num_col=(unpack('>i',num_col_str))[0]    
        print magic_num
        print num_img
        print num_row
        print num_col
        byte = f.read(num_img*num_row*num_col)
    
    byte_c = 0
    for i in range(num_img):
        img = np.zeros((num_row,num_col),dtype=np.int32)
        for h in range(num_row):
            for w in range(num_col):
                img[h,w]=(unpack('>B',byte[byte_c]))[0]
                byte_c+=1
        scipy.misc.imsave(img_save_path+'%d.bmp'%i, img)
    
    print 'open label file'
    with open(img_label_file,'rb') as f_label_in:
        magic_num_str = f_label_in.read(4)
        magic_num = (unpack('>i',magic_num_str))[0]
        num_img_str = f_label_in.read(4)
        num_img=(unpack('>i',num_img_str))[0]
        byte=f_label_in.read(num_img)
    
    print 'magic_num %d' % magic_num
    print 'num_img %d' % num_img
    
    
    with open(out_label_file,'w') as f_label_out:
        labels=np.zeros((num_img),dtype=np.uint32)
        for i in range(num_img):
            labels[i]=(unpack('>B',byte[i]))[0]
            f_label_out.write('%d\n'% labels[i])


In [27]:
extract_images_and_labels(data_dir+train_img_data, data_dir+train_img_label,\
                          caffe_data_dir+train_img_save_dir,caffe_data_dir+'train_labels.txt')
extract_images_and_labels(data_dir+test_img_data, data_dir+test_img_label,\
                          caffe_data_dir+test_img_save_dir,caffe_data_dir+'test_labels.txt')

2051
60000
28
28
open label file
magic_num 2049
num_img 60000
2051
10000
28
28
open label file
magic_num 2049
num_img 10000


In [5]:
def prepare_database(img_data_file,img_label_file, out_database_name):
    out_lmdb = lmdb.open(out_database_name, map_size=1024*1024*1024)
    
    with open(img_data_file,'rb') as f:
        magic_num_str = f.read(4)
        magic_num = (unpack('>i',magic_num_str))[0]
        num_img_str = f.read(4)
        num_img = (unpack('>i',num_img_str))[0]
        num_row_str = f.read(4)
        num_row = (unpack('>i',num_row_str))[0]
        num_col_str = f.read(4)
        num_col = (unpack('>i',num_col_str))[0]  
        pix_byte = f.read(num_img*num_row*num_col)
    print 'num_img %d' % num_img
    print 'num_row %d num_col %d' % (num_row, num_col)
        
    labels = np.zeros((num_img), dtype = np.int32)
    with open(img_label_file, 'rb') as f_label_in:
        magic_num_str = f_label_in.read(4)
        magic_num = (unpack('>i',magic_num_str))[0]
        num_img_str = f_label_in.read(4)
        num_img=(unpack('>i',num_img_str))[0]
        label_byte=f_label_in.read(num_img)
        for i in range(num_img):
            labels[i] = (unpack('>B', label_byte[i]))[0]
    print 'labels'
    print labels[:100]
    img_size = num_row * num_col
    
    batch_size = 500
    num_batch = num_img / batch_size
    byte_c, img_c = 0, 0
    for i in range(num_batch):
        print '%d out of %d batches ' % (i, num_batch)
        with out_lmdb.begin(write=True) as txn:
            for j in range(batch_size):
                datum = caffe_pb2.SemanticLabelingDatum()
                datum.channels = 1
                datum.height = num_row
                datum.width = num_col
                datum.data = pix_byte[img_c * img_size:(img_c + 1) * img_size]
                fg_c = 0
                for h in range(num_row):
                    for w in range(num_col):
                        pix_val = (unpack('>B',pix_byte[byte_c]))[0]
                        if pix_val > 0:
                            datum.label.append(int(labels[img_c]))
                            fg_c += 1
                        else:
                            datum.label.append(10) # background label
                        byte_c += 1
                txn.put('%05d'%img_c, datum.SerializeToString())
                img_c += 1
    
    out_lmdb.close()

In [6]:
# prepare_database(data_dir+train_img_data, data_dir+train_img_label,\
#                 caffe_example_dir+'train_lmdb')
prepare_database(data_dir+test_img_data, data_dir+test_img_label,\
                caffe_example_dir+'test_lmdb')

num_img 10000
num_row 28 num_col 28
labels
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3 7 4 6 4 3 0 7 0 2 9
 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 9 3 1 4 1 7 6 9]
0 out of 20 batches 
1 out of 20 batches 
2 out of 20 batches 
3 out of 20 batches 
4 out of 20 batches 
5 out of 20 batches 
6 out of 20 batches 
7 out of 20 batches 
8 out of 20 batches 
9 out of 20 batches 
10 out of 20 batches 
11 out of 20 batches 
12 out of 20 batches 
13 out of 20 batches 
14 out of 20 batches 
15 out of 20 batches 
16 out of 20 batches 
17 out of 20 batches 
18 out of 20 batches 
19 out of 20 batches 
